In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gstools as gs
from model import PDE_basis, LS_Base

In [2]:
basis_num_vec = [100, 500, 1000]
shape_vec = np.arange(0.2, 5, 0.2)
radius_vec = [1.5]
cor_len_vec = [0.5, 1]

print('basis_num tested:', basis_num_vec)
print('shape tested:', shape_vec)
print('loc_scale tested:', radius_vec)
print('cor_len tested:', cor_len_vec)

basis_num tested: [100, 500, 1000]
shape tested: [0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2.  2.2 2.4 2.6 2.8 3.  3.2 3.4 3.6
 3.8 4.  4.2 4.4 4.6 4.8]
loc_scale tested: [1.5]
cor_len tested: [0.5, 1]


In [3]:
K = 10         # number of GP realizations
x_dim = 2               # fitting dimension
mesh_size = 50   # number of spatial samples in each GP realization

x_1d = np.linspace(-1,1,mesh_size)
x1_2d, x2_2d = np.meshgrid(x_1d,x_1d)
x_train = np.stack([x1_2d.flatten(), x2_2d.flatten()],axis=1)


In [4]:
for cor_len in cor_len_vec:
    print('\ncor_len=', cor_len,end='')
    # get all GP realization
    model = gs.Gaussian(dim=x_dim, var=1, len_scale=cor_len)
    srf = gs.SRF(model)
    srf.set_pos([x_1d, x_1d], "structured")
    # generate fields
    for i in range(K):
        srf(seed=i, store=f"field{i}")

    # LS fitting
    for radius in radius_vec:
        print(f'\n\tradius={radius}:',end='')
        temp_result = np.zeros((len(basis_num_vec), len(shape_vec)))

        for i, basis_num in enumerate(basis_num_vec):
            print(f'\n\t\tbasis_num={basis_num}:',end='')
            # set up basis
            for j, shape in enumerate(shape_vec):
                # set up basis
                print(f' ({shape:.2f})', end='')

                # LS fitting for all GP
                mean_mse = []
                for k in range(K):

                    basis = PDE_basis(x_dim=x_dim, basis_num=basis_num)
                    basis.init_pde_basis(shape=shape, radius=radius)
                    ls_data = {f'GP': [basis.eval_basis(x_train)['u'], srf[k].flatten()[:,None]]}
                    coef, info = LS_Base.ls_fit(data=ls_data, weights=None, ls_mse=True, item_mse=False)
                    ls_mse = info['ls_mse']
                    mean_mse.append(ls_mse)
                mean_mse = np.mean(np.array(mean_mse))
                #
                temp_result[i,j] = mean_mse
        df = pd.DataFrame(temp_result)
        df.columns = [f'shape_{shape}' for shape in shape_vec]
        df.insert (0, 'basis_num', basis_num_vec)
        df.to_csv(f'raw/cor_{cor_len}_r_{radius}_fine.csv',header=True, index=False, encoding='utf-8')



cor_len= 0.5
	radius=1.5:
		basis_num=100: (0.20) (0.40) (0.60) (0.80) (1.00) (1.20) (1.40) (1.60) (1.80) (2.00) (2.20) (2.40) (2.60) (2.80) (3.00) (3.20) (3.40) (3.60) (3.80) (4.00) (4.20) (4.40) (4.60) (4.80)
		basis_num=500: (0.20) (0.40) (0.60) (0.80) (1.00) (1.20) (1.40) (1.60) (1.80) (2.00) (2.20) (2.40) (2.60) (2.80) (3.00) (3.20) (3.40) (3.60) (3.80) (4.00) (4.20) (4.40) (4.60) (4.80)
		basis_num=1000: (0.20) (0.40) (0.60) (0.80) (1.00) (1.20) (1.40) (1.60) (1.80) (2.00) (2.20) (2.40) (2.60) (2.80) (3.00) (3.20) (3.40) (3.60) (3.80) (4.00) (4.20) (4.40) (4.60) (4.80)
cor_len= 1
	radius=1.5:
		basis_num=100: (0.20) (0.40) (0.60) (0.80) (1.00) (1.20) (1.40) (1.60) (1.80) (2.00) (2.20) (2.40) (2.60) (2.80) (3.00) (3.20) (3.40) (3.60) (3.80) (4.00) (4.20) (4.40) (4.60) (4.80)
		basis_num=500: (0.20) (0.40) (0.60) (0.80) (1.00) (1.20) (1.40) (1.60) (1.80) (2.00) (2.20) (2.40) (2.60) (2.80) (3.00) (3.20) (3.40) (3.60) (3.80) (4.00) (4.20) (4.40) (4.60) (4.80)
		basis_num=1000: (0.20